<a href="https://colab.research.google.com/github/SeyfGoumeida/Projet_Apprentissage_supervis-_Cr-dits_bancaires_Donn-es_relationnelles/blob/main/Projet_%C2%AB_Apprentissage_supervis%C3%A9_%C2%BB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prérequirement :
- Imbalanced-Learn Library
- Stellargraph (Cora)

In [1]:
!sudo pip install imbalanced-learn
import imblearn
from imblearn.over_sampling import RandomOverSampler
#Import Gaussian Naive Bayes model
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
#KNN
from sklearn.neighbors import KNeighborsClassifier
#LDA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
#QDA
import sklearn.discriminant_analysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis


#------------------------------------------------
# install StellarGraph if running on Google Colab
import sys
if 'google.colab' in sys.modules:
  %pip install -q stellargraph[demos]==1.0.0rc1
# verify that we're using the correct version of StellarGraph for this notebook
import stellargraph as sg

try:
    sg.utils.validate_notebook_version("1.0.0rc1")
except AttributeError:
    raise ValueError(
        f"This notebook requires StellarGraph version 1.0.0rc1, but a different version {sg.__version__} is installed.  Please see <https://github.com/stellargraph/stellargraph/issues/1172>."
    ) from None
#------------------------------------------------ 
import pandas as pd
import os
import numpy as np
import stellargraph as sg
from stellargraph.mapper import FullBatchNodeGenerator
from stellargraph.layer import GCN
from tensorflow.keras import layers, optimizers, losses, metrics, Model
from sklearn import preprocessing, model_selection
from IPython.display import display, HTML
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from spacy.lang.en.stop_words import STOP_WORDS
import string
import re
import operator
from functools import reduce
import nltk
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
from collections import Counter


     |████████████████████████████████| 374 kB 4.8 MB/s 
     |████████████████████████████████| 482 kB 39.0 MB/s 
     |████████████████████████████████| 41 kB 165 kB/s 
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


#2 - Données relationnelles :

In [12]:
dataset = sg.datasets.Cora()
display(HTML(dataset.description))
G, node_subjects = dataset.load()

In [44]:
print("- La taille du réseau de citations (The citation network) : ",len(G.edges()))
print("- Format du réseau de citations : paper-cites->paper")
print("- Exemple sur une publication et une partie de ses liens :" )
for i in range(0,10):
  print(G.edges()[i])

- La taille du réseau de citations (The citation network) :  5429
- Format du réseau de citations : paper-cites->paper
- Exemple sur une publication et une partie de ses liens :
(1033, 35)
(103482, 35)
(103515, 35)
(1050679, 35)
(1103960, 35)
(1103985, 35)
(1109199, 35)
(1112911, 35)
(1113438, 35)
(1113831, 35)


In [14]:
print(G.info())

StellarGraph: Undirected multigraph
 Nodes: 2708, Edges: 5429

 Node types:
  paper: [2708]
    Features: float32 vector, length 1433
    Edge types: paper-cites->paper

 Edge types:
    paper-cites->paper: [5429]
        Weights: all 1 (default)


In [151]:
node_subjects.value_counts().to_frame()

,subject
Neural_Networks,818
Probabilistic_Methods,426
Genetic_Algorithms,418
Theory,351
Case_Based,298
Reinforcement_Learning,217
Rule_Learning,180


Here we’re taking 140 node labels for training, 500 for validation, and the rest for testing.



In [108]:
train_subjects, test_subjects = model_selection.train_test_split(
    node_subjects, train_size=140, test_size=None, stratify=node_subjects
)
val_subjects, test_subjects = model_selection.train_test_split(
    test_subjects, train_size=500, test_size=None, stratify=test_subjects
)

Note using stratified sampling gives the following counts:



In [109]:
train_subjects.value_counts().to_frame()


,subject
Neural_Networks,42
Genetic_Algorithms,22
Probabilistic_Methods,22
Theory,18
Case_Based,16
Reinforcement_Learning,11
Rule_Learning,9


The training set has class imbalance that might need to be compensated, e.g., via using a weighted cross-entropy loss in model training, with class weights inversely proportional to class support.

In [123]:
#oversample = RandomOverSampler(sampling_strategy='minority')
#oversample = RandomOverSampler(sampling_strategy=0.5)
oversample = RandomOverSampler(random_state=0)



sampling_strategy='minority' => This means that if the majority class had 1,000 examples and the minority class had 100, this strategy would oversampling the minority class so that it has 1,000 examples.

sampling_strategy=0.5 => This would ensure that the minority class was oversampled to have half the number of examples as the majority class, for binary classification problems. This means that if the majority class had 1,000 examples and the minority class had 100, the transformed dataset would have 500 examples of the minority class.




In [142]:
# example of random oversampling to balance the class distribution
# define dataset
print(train_subjects.value_counts().to_frame())
print()
# fit and apply the transform
X_train,y_train = oversample.fit_resample(np.array(list(train_subjects.index)).reshape(-1, 1),train_subjects.values)
# summarize class distribution
X_train,y_train
s = pd.Series(y_train)
s.index = list(X_train.reshape(1,-1))
print(s.value_counts().to_frame())

                        subject
Neural_Networks              42
Genetic_Algorithms           22
Probabilistic_Methods        22
Theory                       18
Case_Based                   16
Reinforcement_Learning       11
Rule_Learning                 9

                         0
Neural_Networks         42
Reinforcement_Learning  42
Rule_Learning           42
Case_Based              42
Theory                  42
Probabilistic_Methods   42
Genetic_Algorithms      42


In [146]:
edge_list = pd.read_csv(
    os.path.join(dataset.data_directory, "cora.cites"),
    sep="\t",
    header=None,
    names=["target", "source"],
)
edge_list["label"] = "cites"
display(edge_list.head(10))


,target,source,label
0,35,1033,cites
1,35,103482,cites
2,35,103515,cites
3,35,1050679,cites
4,35,1103960,cites
5,35,1103985,cites
6,35,1109199,cites
7,35,1112911,cites
8,35,1113438,cites
9,35,1113831,cites


In [157]:
feature_names = ["w_{}".format(ii) for ii in range(1433)]
column_names = feature_names + ["subject"]
node_list = pd.read_csv(
    os.path.join(dataset.data_directory, "cora.content"),
    sep="\t",
    header=None,
    names=column_names,
)


In [158]:
node_list.head(5)


,w_0,w_1,w_2,w_3,w_4,w_5,w_6,w_7,w_8,w_9,w_10,w_11,w_12,w_13,w_14,w_15,w_16,w_17,w_18,w_19,w_20,w_21,w_22,w_23,w_24,w_25,w_26,w_27,w_28,w_29,w_30,w_31,w_32,w_33,w_34,w_35,w_36,w_37,w_38,w_39,...,w_1394,w_1395,w_1396,w_1397,w_1398,w_1399,w_1400,w_1401,w_1402,w_1403,w_1404,w_1405,w_1406,w_1407,w_1408,w_1409,w_1410,w_1411,w_1412,w_1413,w_1414,w_1415,w_1416,w_1417,w_1418,w_1419,w_1420,w_1421,w_1422,w_1423,w_1424,w_1425,w_1426,w_1427,w_1428,w_1429,w_1430,w_1431,w_1432,subject
31336,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,Neural_Networks
1061127,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,Rule_Learning
1106406,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,Reinforcement_Learning
13195,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Reinforcement_Learning
37879,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Probabilistic_Methods


In [159]:
# gather all features into lists under 'features' column.
node_list["features"] = node_list[feature_names].values.tolist()

node_list = node_list.drop(columns=feature_names)
node_list["id"] = node_list.index
node_list.head(5)


,subject,features,id
31336,Neural_Networks,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",31336
1061127,Rule_Learning,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...",1061127
1106406,Reinforcement_Learning,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1106406
13195,Reinforcement_Learning,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",13195
37879,Probabilistic_Methods,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",37879


In [161]:
#The different subjects are:
set(node_list["subject"])


{'Case_Based',
 'Genetic_Algorithms',
 'Neural_Networks',
 'Probabilistic_Methods',
 'Reinforcement_Learning',
 'Rule_Learning',
 'Theory'}

In [220]:
X = np.array(list(node_list['features']))
y = np.array(list(node_list['subject']))


# Data Balancing :

In [250]:
#oversample = RandomOverSampler(sampling_strategy='minority')
#oversample = RandomOverSampler(sampling_strategy=0.5)
oversample = RandomOverSampler(random_state=0)
ydf = pd.DataFrame(y)
print("Avant Data Balancing : ")
print(ydf.value_counts())
print()
# fit and apply the transform
X_train_balanced,y_train_balanced = oversample.fit_resample(X,y)
# summarize class distribution
ydf = pd.DataFrame(y_train_balanced)
print("Aprés Data Balancing : ")
print(ydf.value_counts())

Avant Data Balancing : 
Neural_Networks           818
Probabilistic_Methods     426
Genetic_Algorithms        418
Theory                    351
Case_Based                298
Reinforcement_Learning    217
Rule_Learning             180
dtype: int64

Aprés Data Balancing : 
Theory                    818
Rule_Learning             818
Reinforcement_Learning    818
Probabilistic_Methods     818
Neural_Networks           818
Genetic_Algorithms        818
Case_Based                818
dtype: int64


# Data split :

In [251]:
train_subjects, test_subjects = model_selection.train_test_split(
    node_subjects, train_size=140, test_size=None, stratify=node_subjects
)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=0) # 70% training and 30% test

#--------------------------------------------------------------------------------------------------------------------

train_subjects, test_subjects = model_selection.train_test_split(
    node_subjects, train_size=140, test_size=None, stratify=node_subjects
)
X_train_balanced, X_test_balanced, y_train_balanced, y_test_balanced = train_test_split(X_train_balanced, y_train_balanced, test_size=0.2,random_state=0) # 70% training and 30% test


# Label encoder :

In [240]:
# Import LabelEncoder
from sklearn import preprocessing
#creating labelEncoder
le = preprocessing.LabelEncoder()
# Converting string labels into numbers.
y_encoded=le.fit_transform(y)
print(y_encoded)

[2 5 4 ... 1 0 2]


# Gaussian Naive Bayes :

In [262]:
#Create a Gaussian Classifier
gnb = GaussianNB()

#Train the model using the training sets
gnb.fit(X_train, y_train)

#Predict the response for test dataset
y_pred = gnb.predict(X_test)

print("Number of mislabeled points out of a total %d points : %d"
      % (X_test.shape[0], (y_test != y_pred).sum()))

# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Number of mislabeled points out of a total 542 points : 261
Accuracy: 0.518450184501845


With balanced data :

In [263]:
#Create a Gaussian Classifier
gnb = GaussianNB()

#Train the model using the training sets
gnb.fit(X_train_balanced, y_train_balanced)

#Predict the response for test dataset
y_pred_balanced = gnb.predict(X_test_balanced)


print("Number of mislabeled points out of a total %d points : %d"
      % (X_test_balanced.shape[0], (y_test_balanced != y_pred_balanced).sum()))
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test_balanced, y_pred_balanced))

Number of mislabeled points out of a total 1146 points : 243
Accuracy: 0.7879581151832461


# KNN :

In [283]:
modelknn = KNeighborsClassifier(n_neighbors=3)
# Train the model using the training sets
modelknn.fit(X_train, y_train)
#Predict Output
y_pred= modelknn.predict(X_test) # 0:Overcast, 2:Mild

print("Number of mislabeled points out of a total %d points : %d"
      % (X_test.shape[0], (y_test != y_pred).sum()))
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Number of mislabeled points out of a total 542 points : 302
Accuracy: 0.44280442804428044


With balanced data :

In [284]:
modelknn = KNeighborsClassifier(n_neighbors=3)
# Train the model using the training sets
modelknn.fit(X_train_balanced, y_train_balanced)
#Predict Output
y_pred_balanced= modelknn.predict(X_test_balanced) 

print("Number of mislabeled points out of a total %d points : %d"
      % (X_test_balanced.shape[0], (y_test_balanced != y_pred_balanced).sum()))
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test_balanced, y_pred_balanced))

Number of mislabeled points out of a total 1146 points : 260
Accuracy: 0.7731239092495636


# LDA : 

In [285]:
modelLDA = LinearDiscriminantAnalysis()
modelLDA.fit(X_train, y_train)
y_pred= modelLDA.predict(X_test)

print("Number of mislabeled points out of a total %d points : %d"
      % (X_test.shape[0], (y_test != y_pred).sum()))
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Number of mislabeled points out of a total 542 points : 200
Accuracy: 0.6309963099630996


With balanced data :

In [286]:
modelLDA = LinearDiscriminantAnalysis()
modelLDA.fit(X_train_balanced, y_train_balanced)
y_pred_balanced= modelLDA.predict(X_test_balanced)

print("Number of mislabeled points out of a total %d points : %d"
      % (X_test_balanced.shape[0], (y_test_balanced != y_pred_balanced).sum()))
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test_balanced, y_pred_balanced))

Number of mislabeled points out of a total 1146 points : 135
Accuracy: 0.8821989528795812


# QDA :

In [290]:
modelQDA = QuadraticDiscriminantAnalysis()
modelQDA.fit(X_train, y_train)
y_pred= modelQDA.predict(X_test)

print("Number of mislabeled points out of a total %d points : %d"
      % (X_test.shape[0], (y_test != y_pred).sum()))
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

/usr/local/lib/python3.7/dist-packages/sklearn/discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


Number of mislabeled points out of a total 542 points : 463
Accuracy: 0.14575645756457564


In [291]:
modelQDA = QuadraticDiscriminantAnalysis()
modelQDA.fit(X_train_balanced, y_train_balanced)
y_pred_balanced= modelQDA.predict(X_test_balanced)

print("Number of mislabeled points out of a total %d points : %d"
      % (X_test_balanced.shape[0], (y_test_balanced != y_pred_balanced).sum()))
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test_balanced, y_pred_balanced))

/usr/local/lib/python3.7/dist-packages/sklearn/discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


Number of mislabeled points out of a total 1146 points : 138
Accuracy: 0.8795811518324608


#1 - Données Crédits bancaires :

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/jbrownlee/Datasets/master/german.csv',header=None)
df.rename({0: "Statut du compte courant existant", 1:"Durée en mois",2: "Historique de crédit",3: "Objectif",
           4:"Montant du crédit", 5:"Compte d'épargne", 6:"Emploi actuel depuis", 7:"Taux de versement en",
           8:"pourcentage du revenu disponible", 9:"Statut personnel et sexe",10: "Autres débiteurs",
           11:"Résidence actuelle depuis",12:"Propriété", 13:"Age en années", 14:"Autres plans de versement", 
           15:"Logement",16:"Nombre de crédits existants dans cette banque", 17:"Travail",
           18:"Nombre de personnes à charge", 19:"Téléphone", 20:"Travailleur étranger"}, axis='columns')

,Statut du compte courant existant,Durée en mois,Historique de crédit,Objectif,Montant du crédit,Compte d'épargne,Emploi actuel depuis,Taux de versement en,pourcentage du revenu disponible,Statut personnel et sexe,Autres débiteurs,Résidence actuelle depuis,Propriété,Age en années,Autres plans de versement,Logement,Nombre de crédits existants dans cette banque,Travail,Nombre de personnes à charge,Téléphone,Travailleur étranger
0,A11,6,A34,A43,1169,A65,A75,4,A93,A101,4,A121,67,A143,A152,2,A173,1,A192,A201,1
1,A12,48,A32,A43,5951,A61,A73,2,A92,A101,2,A121,22,A143,A152,1,A173,1,A191,A201,2
2,A14,12,A34,A46,2096,A61,A74,2,A93,A101,3,A121,49,A143,A152,1,A172,2,A191,A201,1
3,A11,42,A32,A42,7882,A61,A74,2,A93,A103,4,A122,45,A143,A153,1,A173,2,A191,A201,1
4,A11,24,A33,A40,4870,A61,A73,3,A93,A101,4,A124,53,A143,A153,2,A173,2,A191,A201,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,A14,12,A32,A42,1736,A61,A74,3,A92,A101,4,A121,31,A143,A152,1,A172,1,A191,A201,1
996,A11,30,A32,A41,3857,A61,A73,4,A91,A101,4,A122,40,A143,A152,1,A174,1,A192,A201,1
997,A14,12,A32,A43,804,A61,A75,4,A93,A101,4,A123,38,A143,A152,1,A173,1,A191,A201,1
998,A11,45,A32,A43,1845,A61,A73,4,A93,A101,4,A124,23,A143,A153,1,A173,1,A192,A201,2
